# Python For Finance - Machine Learning Intro

### WARNING - THIS IS QUANTOPIAN CODE AND MIGHT NOT RUN

## Original @ [PFF14]

In [ ]:
def initialize(context):
    context.features = {}
    context.stocks = symbols('XLY',  # XLY Consumer Discrectionary SPDR Fund   
                           'XLF',  # XLF Financial SPDR Fund  
                           'XLK',  # XLK Technology SPDR Fund  
                           'XLE',  # XLE Energy SPDR Fund  
                           'XLV',  # XLV Health Care SPRD Fund  
                           'XLI',  # XLI Industrial SPDR Fund  
                           'XLP',  # XLP Consumer Staples SPDR Fund   
                           'XLB',  # XLB Materials SPDR Fund  
                           'XLU')  # XLU Utilities SPRD Fund
    
    
def rebalance(context, data):
    pass
  
    

def handle_data(context, data):
    for stock in context.stocks:
        ma1 = data[stock].mavg(100) 
        ma2 = data[stock].mavg(300)
        
        price = data[stock].price
        
        if ma1 > ma2:
            order_target_percent(stock, 0.11)
            
        elif ma1 < ma2:
            order_target_percent(stock, -0.11)  
        
        record('ma1',ma1)
        record('ma2',ma2)
        record('Leverage',context.account.leverage)

## Changes @ [PFF14]

Machine learning has features and labels
We will go through the last 100 days of data.
We will then use 10 day slices to train the program to predict stock prices.

In [ ]:
#add to initialize
context.historical_bars = 100
context.feature_window = 10

In [ ]:
#add to handle_data
prices = history(bar_count = context.historical_bars, frequency='1d', field='price')

In [ ]:
#add to for
start_bar = context.feature_window

Next we convert the last 100 days of prices to a list

In [ ]:
price_list = prices[stock].tolist()

x = []
y = []

## Final @ [PFF14]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from collections import Counter
import numpy as np


def initialize(context):

    context.stocks = symbols('XLY',  # XLY Consumer Discrectionary SPDR Fund   
                           'XLF',  # XLF Financial SPDR Fund  
                           'XLK',  # XLK Technology SPDR Fund  
                           'XLE',  # XLE Energy SPDR Fund  
                           'XLV',  # XLV Health Care SPRD Fund  
                           'XLI',  # XLI Industrial SPDR Fund  
                           'XLP',  # XLP Consumer Staples SPDR Fund   
                           'XLB',  # XLB Materials SPDR Fund  
                           'XLU')  # XLU Utilities SPRD Fund
    
    context.historical_bars = 100
    context.feature_window = 10
    
def handle_data(context, data):
    prices = history(bar_count = context.historical_bars, frequency='1d', field='price')

    for stock in context.stocks:   
        ma1 = data[stock].mavg(50)
        ma2 = data[stock].mavg(200)
        
        start_bar = context.feature_window
        price_list = prices[stock].tolist()
        
        X = []
        y = []


## [PFF15]

### Creating Feature Sets

If price goes up, 1
If price goes down, -1 or 0

In [ ]:
import numpy as np

In [ ]:
#add to for
bar = start_bar

#len = length
#when slicing lists the first element is a 0; thus the -1
while bar < len(price_list)-1:
    try:
        #say price is 192. We want to see the pattern changes of the last 10 changes to find the next price
        end_price = price_list[bar+1]
        start_price = price_list[bar] #the last price on the list
        
        pricing_list = []
        xx = 0
        for _ in range(context.feature_window): #underscore in a for loop means the variable is useless
            price=price_list[bar-(context.feature_window-xx)] # the massive list of prices
            pricing_list.append(price)
            xx += 1 #references the current bar and does the previous prices (bar - 1)
        
        #normalize the data
        #percent change (new-old/old*100)
        features = np.around(np.diff(pricing_list) / pricing_list[:-1] * 100.0, 1)
        
        print(features)
        bar += 1
        
    except Exception as e:
        print(('feature creation',str(e)))

Say the list is: [4,4,5,6,34,33] We want to find the NEXT price on the list. We want the machine learning to "think" and figure out what the next price will be

## Final @ [PFF15]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from collections import Counter
import numpy as np


def initialize(context):

    context.stocks = symbols('XLY',  # XLY Consumer Discrectionary SPDR Fund   
                           'XLF',  # XLF Financial SPDR Fund  
                           'XLK',  # XLK Technology SPDR Fund  
                           'XLE',  # XLE Energy SPDR Fund  
                           'XLV',  # XLV Health Care SPRD Fund  
                           'XLI',  # XLI Industrial SPDR Fund  
                           'XLP',  # XLP Consumer Staples SPDR Fund   
                           'XLB',  # XLB Materials SPDR Fund  
                           'XLU')  # XLU Utilities SPRD Fund
    
    context.historical_bars = 100
    context.feature_window = 10
    

   

def handle_data(context, data):
    prices = history(bar_count = context.historical_bars, frequency='1d', field='price')

    for stock in context.stocks:   
        ma1 = data[stock].mavg(50)
        ma2 = data[stock].mavg(200)
        
        start_bar = context.feature_window
        price_list = prices[stock].tolist()
        
        X = []
        y = []
        
        bar = start_bar
        
        # feature creation
        while bar < len(price_list)-1:
            try:
                end_price = price_list[bar+1]
                begin_price = price_list[bar]
                
                pricing_list = []
                xx = 0
                for _ in range(context.feature_window):
                    price = price_list[bar-(context.feature_window-xx)]
                    pricing_list.append(price)
                    xx += 1
                    
                features = np.around(np.diff(pricing_list) / pricing_list[:-1] * 100.0, 1)
                
                if end_price > begin_price:
                    label = 1
                else:
                    label = -1

                bar += 1
                print(features)

            except Exception as e:
                bar += 1
                print(('feature creation',str(e)))